## Проект по DA

Данные - https://www.kaggle.com/usdot/flight-delays 

Описание - Сводная информация о количестве своевременных, задержанных, отмененных и измененных рейсов в США за 2015 год (5.8 млн записей)


### Задачи:
1. Найти аэропорт с минимальной задержкой вылета 

2. Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

3. Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

4. Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика),  в зависимости от аэропорта вылета  

In [1]:
import pandas as pd
import datetime
import numpy as np 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [2]:
# Загружаем данные, знакомимся с даными, ищем поля для объединения таблиц
df_airlines = pd.read_csv('airlines.csv')
df_airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [3]:
df_airports = pd.read_csv('airports.csv')
df_airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [4]:
df_flights = pd.read_csv('flights.csv', low_memory=False)
df_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# объединяем датафреймы в один
flights = df_flights.merge(df_airports[['IATA_CODE']],how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop('IATA_CODE',axis=1)
flights = df_flights.merge(df_airports[['IATA_CODE']],how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop('IATA_CODE',axis=1)

In [6]:
# Посмотрим все атрибуты таблицы flights
list(flights.columns)

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

In [7]:
flights.loc[:,'SCHEDULED_DEPARTURE':'CANCELLATION_REASON'].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON
0,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN
1,45,41.0,-4.0,17.0,58.0,204.0,194.0,173.0,1448,451.0,4.0,509,455.0,-14.0,0,0,NaN
2,45,31.0,-14.0,25.0,56.0,210.0,200.0,171.0,1448,447.0,4.0,515,451.0,-24.0,0,0,NaN
3,115,107.0,-8.0,25.0,132.0,213.0,218.0,186.0,1533,538.0,7.0,548,545.0,-3.0,0,0,NaN
4,135,NaN,NaN,NaN,NaN,205.0,NaN,NaN,1448,NaN,NaN,600,NaN,NaN,0,1,A


In [8]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332914 entries, 0 to 5332913
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [9]:
# заменим поля на строковый тип
flights[['ORIGIN_AIRPORT','DESTINATION_AIRPORT']] = flights[['ORIGIN_AIRPORT','DESTINATION_AIRPORT']].astype(str)

In [10]:
# ознакомимся со статистичесскими данными
flights.describe(include=int)

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL,DIVERTED,CANCELLED
count,5332914.0,5.332914e+06,5.332914e+06,5.332914e+06,5.332914e+06,5.332914e+06,5.332914e+06,5.332914e+06,5.332914e+06,5.332914e+06
mean,2015.0,6.207210e+00,1.568874e+01,3.919179e+00,2.178419e+03,1.330107e+03,8.228955e+02,1.493747e+03,2.691587e-03,1.639441e-02
std,0.0,3.383807e+00,8.774687e+00,1.993635e+00,1.760521e+03,4.843130e+02,6.077992e+02,5.079263e+02,5.181064e-02,1.269868e-01
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.100000e+01,1.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,3.000000e+00,8.000000e+00,2.000000e+00,7.310000e+02,9.170000e+02,3.730000e+02,1.110000e+03,0.000000e+00,0.000000e+00
50%,2015.0,6.000000e+00,1.600000e+01,4.000000e+00,1.690000e+03,1.325000e+03,6.490000e+02,1.520000e+03,0.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.300000e+01,6.000000e+00,3.252000e+03,1.730000e+03,1.065000e+03,1.919000e+03,0.000000e+00,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,4.983000e+03,2.400000e+03,1.000000e+00,1.000000e+00


In [11]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AS,108,N309AS,ANC,SEA,45,...,455.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,DL,1560,N3743H,ANC,SEA,45,...,451.0,-24.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AS,130,N457AS,FAI,SEA,115,...,545.0,-3.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN


### 1. Найти аэропорт с минимальной задержкой вылета

In [12]:
# Группируем датасет по компаниям и ссумируем общее время задержки 
delay = flights.groupby('AIRLINE')['DEPARTURE_DELAY'].sum().sort_values(ascending=True).reset_index()

In [13]:
# заменяем названия столбца и соединяем таблицы для вывода полного названия авиакомпании
delay.rename(columns = {'AIRLINE':'IATA_CODE'}, inplace = True)
delay = delay.merge(df_airlines, on='IATA_CODE')

In [14]:
print('Aэропорт с минимальной задержкой вылета:', 
      delay.iloc[0][2],
      delay.iloc[0][1])

Aэропорт с минимальной задержкой вылета: Hawaiian Airlines Inc. 35923.0


### 2. Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [15]:
# найдем код аэропорта Лос Анджелеса
city = df_airports.loc[df_airports['CITY'] == 'Los Angeles']
city


,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
176,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807


In [16]:
# формируем датасет, отбираем записи только с прилетом в Лос Анджелеса, группируем по авиакомпаниям
best_punctuation = flights[flights['DESTINATION_AIRPORT'] == city.iloc[0][0]]\
                .groupby('AIRLINE')['ARRIVAL_DELAY']\
                .sum()\
                .sort_values(ascending=True)\
                .reset_index()

In [17]:
# заменяем названия столбца и соединяем таблицы для вывода полного названия авиакомпании
best_punctuation.rename(columns = {'AIRLINE':'IATA_CODE'}, inplace = True)
best_punctuation = best_punctuation.merge(df_airlines, how='outer', on='IATA_CODE' )

In [18]:
print('Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport:', 
      best_punctuation.iloc[1][2],
      best_punctuation.iloc[1][1])


Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport: JetBlue Airways 629.0


### 3. Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [19]:
df_plan = flights.groupby('DESTINATION_AIRPORT')['TAXI_OUT'].mean().sort_values(ascending=False).reset_index()

In [20]:
df_plan.rename(columns = {'DESTINATION_AIRPORT':'IATA_CODE'}, inplace = True)
df_plan = df_plan.merge(df_airports, how='left', on='IATA_CODE' )

In [21]:
print('Aэропорт, где самолёты проводят больше всего времени на рулении (среднее значение):', 
      df_plan.iloc[0][2],
      df_plan.iloc[0][1])

Aэропорт, где самолёты проводят больше всего времени на рулении (среднее значение): Southwest Oregon Regional Airport (North Bend Municipal) 23.858736059479554


### 4. Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

In [22]:
df = flights.copy()

In [23]:
# функция создания столбца с датой
def make_date(df):
  return datetime.date(df.YEAR, df.MONTH, df.DAY)

In [24]:
# добавляем столбец с датой
df['dDate'] = df.apply(make_date,axis = 1)

In [25]:
# группируем датасет по дате и аэропорту прилета и суммируем время задержки
df_grouped = df.groupby(['dDate','DESTINATION_AIRPORT'])['ARRIVAL_DELAY'].sum().reset_index()

In [26]:
# приводим столбец 'dDate' в тип данных datetime
df_grouped['dDate'] = pd.to_datetime(df_grouped['dDate'])
df_grouped.head()

,dDate,DESTINATION_AIRPORT,ARRIVAL_DELAY
0,2015-01-01,ABE,-34.0
1,2015-01-01,ABI,0.0
2,2015-01-01,ABQ,654.0
3,2015-01-01,ABR,-19.0
4,2015-01-01,ABY,-29.0


In [27]:
# функция для создания признаков, с учетом задержки предыдущих дней
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.dDate.dt.year
    data['month'] = data.dDate.dt.month
    data['day'] = data.dDate.dt.day
    data['dayofweek'] = data.dDate.dt.dayofweek
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)

In [28]:
# для каждого аэропорта прибытия циклом создадим датасет
where_to_go = []
for dest in df_grouped.DESTINATION_AIRPORT.unique():
    tempo = df_grouped[df_grouped.DESTINATION_AIRPORT==dest][['dDate','ARRIVAL_DELAY']]
    tempo.columns = ['dDate','ARRIVAL_DELAY']
    tempo = pd.DataFrame(tempo)
    
    try:
    
        make_features(tempo,21,7) # создадам фичи
        tempo.dropna(inplace=True) # удалим пустые значения
        tempo.set_index('dDate',inplace=True)  # приведем к DataFrame
           
        # делим на тренировочные и тестовые данные
        X_train, X_test, y_train, y_test = train_test_split(tempo.drop('ARRIVAL_DELAY', axis=1),tempo.ARRIVAL_DELAY, shuffle=False, test_size=0.25)
        # создаем модель линейной регрессии и обучаем модель
        model_lr = LinearRegression()
        model_lr.fit(X_train,y_train)
        
        # предсказываем значения
        y_predicted_lr = model_lr.predict(X_test)

        # добавляем в лист where_to_go аэропорт, среднее время задержки по тестовым данным и rmse
        where_to_go.append([dest,y_test.mean(),np.sqrt(mean_squared_error(y_test, y_predicted_lr))])

    except Exception as e:
        print('Error', str(e))
        

In [29]:
# приводим лист where_to_go к DataFrame и даем названия колонкам
where_to_go = pd.DataFrame(where_to_go)
where_to_go.columns = ['DESTINATION_AIRPORT','MEAN_ARRIVAL_DELAY_IN_PAST','RMSE']
where_to_go.head()

,DESTINATION_AIRPORT,MEAN_ARRIVAL_DELAY_IN_PAST,RMSE
0,ABE,32.316456,96.974756
1,ABI,-5.683544,92.460573
2,ABQ,205.240506,410.706845
3,ABR,0.443038,44.048350
4,ABY,26.645570,64.908040


In [30]:
start_airport = input("Выберите аэропорт вылета: ")
finish_airoport = flights[flights.ORIGIN_AIRPORT==start_airport]['DESTINATION_AIRPORT'].unique()

finish_airoport = pd.DataFrame(finish_airoport)
finish_airoport.columns = ['DESTINATION_AIRPORT']

best_time = finish_airoport.merge(where_to_go,on='DESTINATION_AIRPORT',how='inner')\
                           .sort_values(by=['RMSE','MEAN_ARRIVAL_DELAY_IN_PAST'],ascending=[True, True])\
                           .head(3)

print('Аэропорты с минимальной вероятностью опоздания:')
print('')
print(best_time.reset_index())

Выберите аэропорт вылета: ABQ
Аэропорты с минимальной вероятностью опоздания:

   index DESTINATION_AIRPORT  MEAN_ARRIVAL_DELAY_IN_PAST        RMSE
0      7                 MCI                  138.240506  856.259591
1      6                 PDX                   53.746835  927.363058
2     18                 OAK                  639.253165  933.528152
